In [2]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle

# Make sdf from parcel master

In [3]:

filePath = "F:/GIS/PARCELUPDATE/Workspace/"
sdeEdit    = os.path.join(filePath, "Edit.sde")
# parcel feature class years to use for all cumulative accounting data
years = [2012, 2018, 2019, 2020, 2021, 2022, 2023]

# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
# filter to years
sdf_units = sdf_units.loc[sdf_units['YEAR'].isin(years)]

c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [3]:
# Bring in demographic data for households and population
sr = arcpy.SpatialReference(26910)
# block group feature layer polygons
block_groups_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'
sdf_census_geo = get_fs_data_spatial(block_groups_url)
sdf_block = sdf_census_geo.loc[(sdf_census_geo['YEAR'] == 2020) & (sdf_census_geo['GEOGRAPHY'] == 'Block Group')]
sdf_block.spatial.sr = sr
sdf_tract = sdf_census_geo.loc[(sdf_census_geo['YEAR'] == 2020) & (sdf_census_geo['GEOGRAPHY'] == 'Tract')]
sdf_tract.spatial.sr = sr

c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\arcgis\features\geo\_accessor.py:1527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[col] = array
c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\arcgis\features\geo\_accessor.py:1527: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[col] = array


In [1]:
# bring in demogrpahic data
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/28'
df_census = get_fs_data(census_url)

NameError: name 'get_fs_data' is not defined

# Generate a list of variables and geographies that we want to summarize at the parcel level

In [ ]:
import numpy as np

def calculate_occupancy_rates(df, year, occupancy_vars):
    """
    Calculate occupancy and seasonal rates at the block group level.

    Parameters:
    df (pd.DataFrame): Census data with 'TRPAID', 'variable_code', 'value', and 'Year' columns.
    year (int): The year of interest.
    occupancy_vars (dict): A dictionary mapping descriptive variable names to census codes.
                           Example: {
                               'vacant_units': 'B25002_003E',
                               'occupied_units': 'B25002_002E',
                               'seasonal_units': 'B25004_006E'
                           }

    Returns:
    pd.DataFrame: DataFrame with occupancy and seasonal rates.
    """
    
    # Filter data for selected year and census codes
    df_filtered = df.query("variable_code in @occupancy_vars.values() and year_sample == @year")
    
    # Pivot to wide format
    df_pivot = df_filtered.pivot(index='TRPAID', columns='variable_code', values='value')
    
    # Rename columns based on the provided dictionary
    df_pivot = df_pivot.rename(columns={v: k for k, v in occupancy_vars.items()})

    # Calculate total units
    df_pivot['total_units'] = df_pivot['vacant_units'] + df_pivot['occupied_units']

    # Avoid division by zero
    df_pivot['occupancy_rate'] = df_pivot['occupied_units'] / df_pivot['total_units'].replace(0, np.nan)
    df_pivot['seasonal_rate'] = df_pivot['seasonal_units'] / df_pivot['total_units'].replace(0, np.nan)

    # Reset index for clarity
    return df_pivot.reset_index()


In [ ]:
occupancy_codes = {
    'vacant_units': 'B25002_003E',
    'occupied_units': 'B25002_002E',
    'seasonal_units': 'B25004_006E',
    'household_size': 'B25010_001E'
}
# We should add population
df_census = df_census.loc[df_census['sample_level'] == 'block group']
df_occupancy = calculate_occupancy_rates(df_census, 2023, occupancy_codes)

In [ ]:
with arcpy.EnvManager(outputZFlag="Disabled"):
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features=r"F:\GIS\DB_CONNECT\Vector.sde\SDE.Census\SDE.Tahoe_Census_Geography",
        Output_Geodatabase=r"C:\GIS\Scratch.gdb"
    )

sdf_block_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/Tahoe_Census_Geography')
sdf_block_noz = sdf_block_noz.loc[(sdf_block_noz['YEAR'] == 2020) & (sdf_block_noz['GEOGRAPHY'] == 'Block Group')]
sdf_block_noz.spatial.sr = sr

sdf_tract_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/Tahoe_Census_Geography')
sdf_tract_noz = sdf_tract_noz.loc[(sdf_tract_noz['YEAR'] == 2020) & (sdf_tract_noz['GEOGRAPHY'] == 'Tract')]    
sdf_tract_noz.spatial.sr = sr

In [ ]:
sdf_tract_noz = pd.DataFrame.spatial.from_featureclass('C:/GIS/Scratch.gdb/Tahoe_Census_Geography')
sdf_tract_noz = sdf_tract_noz.loc[(sdf_tract_noz['YEAR'] == 2020) & (sdf_tract_noz['GEOGRAPHY'] == 'Tract')]    
sdf_tract_noz.spatial.sr = sr

In [ ]:
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'

In [ ]:
# spatial join to get Block Group
arcpy.SpatialJoin_analysis(sdf_units, sdf_block_noz, "Existing_Development_BlockGroup", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Tract
arcpy.SpatialJoin_analysis(sdf_units, sdf_tract_noz, "Existing_Development_Tract", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")

In [ ]:

sdf_units_block = pd.DataFrame.spatial.from_featureclass("Existing_Development_BlockGroup", sr=sr)
sdf_units_tract = pd.DataFrame.spatial.from_featureclass("Existing_Development_Tract", sr=sr)

In [ ]:
sdfParcel = sdf_units

In [ ]:
sdfParcel['BLOCK_GROUP']   = sdfParcel.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
sdfParcel['TRACT']        = sdfParcel.APN.map(dict(zip(sdf_units_tract.APN, sdf_units_tract.TRPAID)))

In [ ]:
sdfParcel.to_pickle('parcel_pickle_part1.pkl')

In [ ]:
sdfParcel_2023 = sdfParcel.loc[sdfParcel['YEAR'] == 2023]

In [ ]:
sdfParcel_2023['occupied_rate'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_occupancy.TRPAID, df_occupancy.occupancy_rate)))
sdfParcel_2023['seasonal_rate'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_occupancy.TRPAID, df_occupancy.seasonal_rate)))
sdfParcel_2023['household_size'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_occupancy.TRPAID, df_occupancy.household_size)))

In [ ]:
sdfParcel_2023['occupied_units']=   sdfParcel_2023['occupied_rate'] * sdfParcel_2023['Residential_Units']
sdfParcel_2023['seasonal_units']=  sdfParcel_2023['seasonal_rate'] * sdfParcel_2023['Residential_Units']
sdfParcel_2023['num_of_residents'] = sdfParcel_2023['household_size'] * sdfParcel_2023['occupied_units']

In [ ]:
tesselation = get_fs_data_spatial('https://maps.trpa.org/server/rest/services/Transportation_Equity_Analysis_Tessellation/FeatureServer/0')

In [ ]:
# spatial join to get GRID_ID
arcpy.SpatialJoin_analysis(sdf_units, tesselation, "Existing_Development_Hex", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")

In [ ]:
sdf_units_hex = pd.DataFrame.spatial.from_featureclass("Existing_Development_Hex", sr=sr)
sdfParcel['GRID_ID']        = sdfParcel.APN.map(dict(zip(sdf_units_tract.APN, sdf_units_tract.TRPAID)))

In [ ]:
sdfParcel_2023['GRID_ID'] = sdfParcel_2023.APN.map(dict(zip(sdf_units_hex.APN, sdf_units_hex.GRID_ID)))

In [ ]:
# group by GRID_ID and sum the values for occupied_units, seasonal_units, and num_of_residents
sdfParcel_2023_grouped = sdfParcel_2023.groupby('GRID_ID').agg({
    'occupied_units': 'sum',
    'seasonal_units': 'sum',
    'num_of_residents': 'sum'
}).reset_index()
sdfParcel_2023_grouped['total_units'] = sdfParcel_2023_grouped['occupied_units'] + sdfParcel_2023_grouped['seasonal_units']
sdfParcel_2023_grouped['occupied_unit_rate'] = sdfParcel_2023_grouped['occupied_units']/sdfParcel_2023_grouped['total_units'].replace(0, np.nan)
sdfParcel_2023_grouped['seasonal_unit_rate'] = sdfParcel_2023_grouped['seasonal_units']/sdfParcel_2023_grouped['total_units'].replace(0, np.nan)

In [ ]:
sdfParcel_2023_grouped['occupied_unit_rate'] = sdfParcel_2023_grouped['occupied_unit_rate'].fillna(0)
sdfParcel_2023_grouped['seasonal_unit_rate'] = sdfParcel_2023_grouped['seasonal_unit_rate'].fillna(0)

In [ ]:
tesselation_attributed = tesselation.merge(sdfParcel_2023_grouped, on='GRID_ID', how='left')
tesselation_attributed['occupied_unit_rate'] = tesselation_attributed['occupied_unit_rate'].fillna(0)
tesselation_attributed['seasonal_unit_rate'] = tesselation_attributed['seasonal_unit_rate'].fillna(0)
tesselation_attributed['occupied_units'] = tesselation_attributed['occupied_units'].fillna(0)
tesselation_attributed['seasonal_units'] = tesselation_attributed['seasonal_units'].fillna(0)
tesselation_attributed['num_of_residents'] = tesselation_attributed['num_of_residents'].fillna(0)
tesselation_attributed['total_units'] = tesselation_attributed['total_units'].fillna(0)

In [ ]:
# export tesselation_attributed to a shapefile correctly
output_path = 'C:\GIS\Scratch.gdb\Tessellation_Attributed'   
arcpy.management.CopyFeatures(tesselation_attributed, output_path)

In [ ]:
# make a feature class from the centroid of tesselation attributed
export_fc = 'C:\GIS\Scratch.gdb\Tessellation_2023'
arcpy.management.FeatureToPoint(tesselation_attributed, export_fc, "CENTROID")

In [47]:
# import from csv
disability_status = pd.read_csv('Lookup_Lists/disability_status.csv')
# drop last character from variable code
#disability_status['variable_code'] = disability_status['variable_code'].str[:-1]
#disability_status['variable_code'] = disability_status['variable_code'].astype(str)
#inner join with census data
df_disability = df_census.merge(disability_status, on='variable_code', how='inner')


In [48]:
df_disability_grouped = df_disability.groupby(['TRPAID', 'Disability']).agg({'value': 'sum'}).reset_index()

In [52]:
df_disability_pivot = df_disability_grouped.pivot(index='TRPAID', columns='Disability', values='value').reset_index()
# rename columns to disabilty_yes and disability_no

# Rename the columns
df_disability_pivot.columns.name = None  # Remove the name from the columns index
df_disability_pivot = df_disability_pivot.rename(columns={True: 'disability_yes', False: 'disability_no'})
# create a new column with the proportion of disability_yes to total
df_disability_pivot['disability_yes'] = df_disability_pivot['disability_yes'].fillna(0)
df_disability_pivot['disability_no'] = df_disability_pivot['disability_no'].fillna(0)   
df_disability_pivot['disability_rate'] = df_disability_pivot['disability_yes']/(df_disability_pivot['disability_no'] + df_disability_pivot['disability_yes']).replace(0, np.nan)
df_disability_pivot['disability_rate'] = df_disability_pivot['disability_rate'].fillna(0)
# merge with tesselation_attributed 

In [53]:
sdfParcel_2023['disability_rate'] = sdfParcel_2023['TRACT'].map(dict(zip(df_disability_pivot.TRPAID, df_disability_pivot.disability_rate)))
sdfParcel_2023['disability_yes'] = sdfParcel_2023['TRACT'].map(dict(zip(df_disability_pivot.TRPAID, df_disability_pivot.disability_yes)))

C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\541034960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['disability_rate'] = sdfParcel_2023['TRACT'].map(dict(zip(df_disability_pivot.TRPAID, df_disability_pivot.disability_rate)))
C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\541034960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['disability_yes'] = sdfParcel_2023['TRACT'].map(dict(zip(df_disability_pivot.TRPAID, df_disability_pivot.disability_yes)))


In [ ]:
df_BIPOC = df_census 

In [ ]:
# import from csv
disability_status = pd.read_csv('Lookup_Lists/disability_status.csv')
# drop last character from variable code
#disability_status['variable_code'] = disability_status['variable_code'].str[:-1]
#disability_status['variable_code'] = disability_status['variable_code'].astype(str)
#inner join with census data
df_disability = df_census.merge(disability_status, on='variable_code', how='inner')
df_disability_grouped = df_disability.groupby(['TRPAID', 'Disability']).agg({'value': 'sum'}).reset_index()
df_disability_pivot = df_disability_grouped.pivot(index='TRPAID', columns='Disability', values='value').reset_index()
# rename columns to disabilty_yes and disability_no

# Rename the columns
df_disability_pivot.columns.name = None  # Remove the name from the columns index
df_disability_pivot = df_disability_pivot.rename(columns={True: 'disability_yes', False: 'disability_no'})
# create a new column with the proportion of disability_yes to total
df_disability_pivot['disability_yes'] = df_disability_pivot['disability_yes'].fillna(0)
df_disability_pivot['disability_no'] = df_disability_pivot['disability_no'].fillna(0)   
df_disability_pivot['disability_rate'] = df_disability_pivot['disability_yes']/(df_disability_pivot['disability_no'] + df_disability_pivot['disability_yes']).replace(0, np.nan)
df_disability_pivot['disability_rate'] = df_disability_pivot['disability_rate'].fillna(0)

In [54]:
import pandas as pd
import numpy as np

def calculate_rate_from_lookup(df_census, lookup_csv, group_col, new_col_names):
    """
    Calculates group-based rates from census data and a lookup CSV.

    Parameters:
        df_census (pd.DataFrame): The main census DataFrame.
        lookup_csv (str): Path to the lookup list CSV.
        group_col (str): Column name from the lookup to group by (e.g., 'Disability').
        new_col_names (dict): Dictionary to rename the boolean columns (e.g., {True: 'disability_yes', False: 'disability_no'}).

    Returns:
        pd.DataFrame: Pivoted DataFrame with rates.
    """
    # Load lookup CSV
    lookup_df = pd.read_csv(lookup_csv)

    # Merge with census data
    df_merged = df_census.merge(lookup_df, on='variable_code', how='inner')

    # Group and aggregate values
    df_grouped = df_merged.groupby(['TRPAID', group_col])['value'].sum().reset_index()

    # Pivot to wide format
    df_pivot = df_grouped.pivot(index='TRPAID', columns=group_col, values='value').reset_index()

    # Clean and rename columns
    df_pivot.columns.name = None
    df_pivot = df_pivot.rename(columns=new_col_names)

    # Fill NaNs with 0
    for col in new_col_names.values():
        if col in df_pivot.columns:
            df_pivot[col] = df_pivot[col].fillna(0)
        else:
            df_pivot[col] = 0  # Add missing columns if needed

    # Calculate rate
    df_pivot['rate'] = df_pivot['bipoc_yes']/df_pivot[list(new_col_names.values())].sum(axis=1)

    return df_pivot


In [58]:
df_census_2023_bg = df_census.loc[df_census['sample_level'] == 'block group']
df_census_2023_bg = df_census_2023_bg.loc[df_census_2023_bg['year_sample'] == 2023]
df_bipoc = calculate_rate_from_lookup(
    df_census_2023_bg,
    'Lookup_Lists/Bipoc_Variables.csv',
    'BIPOC',
    {'Yes': 'bipoc_yes', 'No': 'bipoc_no'}
)

In [75]:
sdfParcel_2023['bipoc_yes'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_bipoc.TRPAID, df_bipoc.bipoc_yes)))
sdfParcel_2023['bipoc_rate'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_bipoc.TRPAID, df_bipoc.rate)))

C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\1185533033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['bipoc_yes'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_bipoc.TRPAID, df_bipoc.bipoc_yes)))
C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\1185533033.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['bipoc_rate'] = sdfParcel_2023['BLOCK_GROUP'].map(dict(zip(df_bipoc.TRPAID, df_bipoc.rate)))


In [77]:
df_chas_housing = pd.read_csv('Dowloaded_Data/chas_housing_issues.csv')
df_chas_housing['TRPAID']= df_chas_housing['TRPAID'].astype(str)
# If trpaid is less than 10 digits, add leading zeros
df_chas_housing['TRPAID'] = df_chas_housing['TRPAID'].str.zfill(15)
#Calculate rates so that they can be multiplied by the number of occupied units
df_chas_housing['housing_issues_total'] = df_chas_housing['Housing_Issues_Yes_Owner'] + df_chas_housing['Housing_Issue_No_Owner'] + df_chas_housing['Housing_Issue_Yes_Renter'] + df_chas_housing['Housing_Issue_No_Renter']
df_chas_housing['owner_issue_rate']= df_chas_housing['Housing_Issues_Yes_Owner']/df_chas_housing['housing_issues_total'].replace(0, np.nan)
df_chas_housing['renter_issue_rate']= df_chas_housing['Housing_Issue_Yes_Renter']/df_chas_housing['housing_issues_total'].replace(0, np.nan)
df_chas_housing['owner_issue_rate'] = df_chas_housing['owner_issue_rate'].fillna(0)
df_chas_housing['renter_issue_rate'] = df_chas_housing['renter_issue_rate'].fillna(0)
sdfParcel_2023['owner_issue_rate'] = sdfParcel_2023['TRACT'].map(dict(zip(df_chas_housing.TRPAID, df_chas_housing.owner_issue_rate)))
sdfParcel_2023['renter_issue_rate'] = sdfParcel_2023['TRACT'].map(dict(zip(df_chas_housing.TRPAID, df_chas_housing.renter_issue_rate)))
sdfParcel_2023['housing_issues_renter'] = sdfParcel_2023['renter_issue_rate'] * sdfParcel_2023['occupied_units']
sdfParcel_2023['housing_issues_owner'] = sdfParcel_2023['owner_issue_rate'] * sdfParcel_2023['occupied_units']

C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\4121358557.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['owner_issue_rate'] = sdfParcel_2023['TRACT'].map(dict(zip(df_chas_housing.TRPAID, df_chas_housing.owner_issue_rate)))
C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\4121358557.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['renter_issue_rate'] = sdfParcel_2023['TRACT'].map(dict(zip(df_chas_housing.TRPAID, df_chas_housing.renter_issue_rate)))
C:\Users

In [ ]:
# going to have to go back and redo all of this for 2021 since that's when the CHAS data is from   

In [76]:
sdfParcel_2023['bipoc_number'] = sdfParcel_2023['bipoc_rate'] * sdfParcel_2023['num_of_residents']
sdfParcel_2023['disability_number'] = sdfParcel_2023['disability_rate'] * sdfParcel_2023['num_of_residents']

C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\259334877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['bipoc_number'] = sdfParcel_2023['bipoc_rate'] * sdfParcel_2023['num_of_residents']
C:\Users\amcclary\AppData\Local\Temp\ipykernel_22148\259334877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdfParcel_2023['disability_number'] = sdfParcel_2023['disability_rate'] * sdfParcel_2023['num_of_residents']


In [79]:
# group by GRID_ID and sum the values for occupied_units, seasonal_units, and num_of_residents
sdfParcel_2023_grouped = sdfParcel_2023.groupby('GRID_ID').agg({
    'occupied_units': 'sum',
    'seasonal_units': 'sum',
    'num_of_residents': 'sum',
    'bipoc_number': 'sum',
    'disability_number': 'sum',
    'housing_issues_renter': 'sum',
    'housing_issues_owner': 'sum',
}).reset_index()
sdfParcel_2023_grouped['total_units'] = sdfParcel_2023_grouped['occupied_units'] + sdfParcel_2023_grouped['seasonal_units']
sdfParcel_2023_grouped['occupied_unit_rate'] = sdfParcel_2023_grouped['occupied_units']/sdfParcel_2023_grouped['total_units'].replace(0, np.nan)
sdfParcel_2023_grouped['seasonal_unit_rate'] = sdfParcel_2023_grouped['seasonal_units']/sdfParcel_2023_grouped['total_units'].replace(0, np.nan)
sdfParcel_2023_grouped['occupied_unit_rate'] = sdfParcel_2023_grouped['occupied_unit_rate'].fillna(0)
sdfParcel_2023_grouped['seasonal_unit_rate'] = sdfParcel_2023_grouped['seasonal_unit_rate'].fillna(0)
sdfParcel_2023_grouped['occupied_units'] = sdfParcel_2023_grouped['occupied_units'].fillna(0)
sdfParcel_2023_grouped['seasonal_units'] = sdfParcel_2023_grouped['seasonal_units'].fillna(0)
sdfParcel_2023_grouped['num_of_residents'] = sdfParcel_2023_grouped['num_of_residents'].fillna(0)
sdfParcel_2023_grouped['bipoc_number'] = sdfParcel_2023_grouped['bipoc_number'].fillna(0)
sdfParcel_2023_grouped['disability_number'] = sdfParcel_2023_grouped['disability_number'].fillna(0)
sdfParcel_2023_grouped['housing_issues_renter'] = sdfParcel_2023_grouped['housing_issues_renter'].fillna(0)
sdfParcel_2023_grouped['housing_issues_owner'] = sdfParcel_2023_grouped['housing_issues_owner'].fillna(0)
sdfParcel_2023_grouped['bipoc_rate'] = sdfParcel_2023_grouped['bipoc_number']/sdfParcel_2023_grouped['num_of_residents'].replace(0, np.nan)
sdfParcel_2023_grouped['disability_rate'] = sdfParcel_2023_grouped['disability_number']/sdfParcel_2023_grouped['num_of_residents'].replace(0, np.nan)
sdfParcel_2023_grouped['housing_issues_rate_renter'] = sdfParcel_2023_grouped['housing_issues_renter']/sdfParcel_2023_grouped['occupied_units'].replace(0, np.nan)
sdfParcel_2023_grouped['housing_issues_rate_owner'] = sdfParcel_2023_grouped['housing_issues_owner']/sdfParcel_2023_grouped['occupied_units'].replace(0, np.nan)
# fillna with 0
sdfParcel_2023_grouped['bipoc_rate'] = sdfParcel_2023_grouped['bipoc_rate'].fillna(0)
sdfParcel_2023_grouped['disability_rate'] = sdfParcel_2023_grouped['disability_rate'].fillna(0)
sdfParcel_2023_grouped['housing_issues_rate_renter'] = sdfParcel_2023_grouped['housing_issues_rate_renter'].fillna(0)
sdfParcel_2023_grouped['housing_issues_rate_owner'] = sdfParcel_2023_grouped['housing_issues_rate_owner'].fillna(0)

In [81]:
tesselation_attributed = tesselation.merge(sdfParcel_2023_grouped, on='GRID_ID', how='left')
tesselation_attributed['occupied_unit_rate'] = tesselation_attributed['occupied_unit_rate'].fillna(0)
tesselation_attributed['seasonal_unit_rate'] = tesselation_attributed['seasonal_unit_rate'].fillna(0)
tesselation_attributed['occupied_units'] = tesselation_attributed['occupied_units'].fillna(0)
tesselation_attributed['seasonal_units'] = tesselation_attributed['seasonal_units'].fillna(0)
tesselation_attributed['num_of_residents'] = tesselation_attributed['num_of_residents'].fillna(0)
tesselation_attributed['total_units'] = tesselation_attributed['total_units'].fillna(0)
tesselation_attributed['bipoc_number'] = tesselation_attributed['bipoc_number'].fillna(0)
tesselation_attributed['disability_number'] = tesselation_attributed['disability_number'].fillna(0)
tesselation_attributed['housing_issues_renter'] = tesselation_attributed['housing_issues_renter'].fillna(0)
tesselation_attributed['housing_issues_owner'] = tesselation_attributed['housing_issues_owner'].fillna(0)
tesselation_attributed['bipoc_rate'] = tesselation_attributed['bipoc_rate'].fillna(0)
tesselation_attributed['disability_rate'] = tesselation_attributed['disability_rate'].fillna(0)
tesselation_attributed['housing_issues_rate_renter'] = tesselation_attributed['housing_issues_rate_renter'].fillna(0)
tesselation_attributed['housing_issues_rate_owner'] = tesselation_attributed['housing_issues_rate_owner'].fillna(0)
# export tesselation_attributed to a shapefile correctly
output_path = 'C:\GIS\Scratch.gdb\Tessellation_Attributed_Full'
arcpy.management.CopyFeatures(tesselation_attributed, output_path)

'C:\\GIS\\Scratch.gdb\\Tessellation_Attributed_Full'

In [82]:
# make a feature class from the centroid of tesselation attributed
export_fc = 'C:\GIS\Scratch.gdb\Tessellation_2023_point'
arcpy.management.FeatureToPoint(tesselation_attributed, export_fc, "CENTROID")

'C:\\GIS\\Scratch.gdb\\Tessellation_2023_point'